<a href="https://colab.research.google.com/github/BEMiklos/PlantTraits2024/blob/main/train_baselines_sceleton_hyperparamOPT_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook contains the híperparameter optimalization of VGG, and the augmentation trials for EfficientNET



# **PlantTraits**

### Imports

In [ ]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install -q kaggle
!pip install -q albumentations

In [ ]:
!pip install -q optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.8 MB/s eta 0:00:00


TODO befor running:Create New API Token and add it as kaggle.json to root

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c planttraits2024

100% 3.17G/3.17G [02:32<00:00, 22.6MB/s]
100% 3.17G/3.17G [02:32<00:00, 22.3MB/s]


In [ ]:
!unzip -q '/content/planttraits2024.zip'

In [ ]:
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A
import cv2

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR
from torchvision.io import read_image
from torch.utils.data import Dataset, ConcatDataset, DataLoader, Subset, random_split
from torchvision import transforms

from sklearn.model_selection import train_test_split

import re

import optuna

### Set the Device

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
random_state = 7

np.random.seed(0)

random.seed(0)

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
TRAIN_IMAGES_PATH = '/content/train_images'
TEST_IMAGES_PATH = '/content/test_images'

### Untility functions

In [ ]:
mean_pattern = r'X\d+_mean'
sd_pattern = r'X\d+_sd'

In [ ]:
def find_matching_elements(lst,pattern):
    matching_elements = [element for element in lst if re.search(pattern, element)]
    return matching_elements

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.sample(10)

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
24886,184873835,-1.032917,574.295227,90.252380,62.482738,1227.480225,42.762859,87,144,125,...,0.174163,0.645791,0.433834,615.749773,NaN,NaN,NaN,NaN,NaN,NaN
31965,190494649,0.440794,589.538086,48.000000,33.024284,1090.136475,39.804287,54,149,118,...,0.554565,1.121493,1.130741,725.767936,0.002047,0.909884,0.073212,0.059998,0.042375,30.941051
28421,195642066,17.284445,341.909515,65.457146,89.490135,544.396545,31.776667,138,163,151,...,0.439074,0.601914,1.030841,304.821680,0.003821,0.326769,0.027901,0.153138,0.013564,16.485387
12674,164426751,12.075297,166.142853,24.500000,49.969826,902.658264,38.064285,129,148,141,...,0.199941,3.176854,2.202036,509.618373,NaN,NaN,NaN,NaN,NaN,NaN
31158,186882920,17.013542,1030.795898,266.556122,91.027039,177.292847,14.822959,107,117,112,...,0.231805,0.051018,1.190324,89.976644,0.005543,0.221099,0.025756,0.024287,0.015604,5.421096
45443,195637432,-0.526925,317.861908,87.438095,113.449104,1096.622192,40.898571,108,146,128,...,0.359241,1.062527,1.332752,3322.765956,NaN,NaN,NaN,NaN,NaN,NaN
19727,194871319,20.958969,2068.119141,264.828583,53.143101,488.990204,19.647144,115,144,128,...,0.232182,0.041521,2.149017,669.891966,NaN,NaN,NaN,NaN,NaN,NaN
20600,186386731,18.402203,563.590454,81.157143,64.466095,437.827026,24.329046,121,136,131,...,0.230897,0.500619,0.942541,608.056653,NaN,NaN,NaN,NaN,NaN,NaN
24052,164361896,17.402231,870.571411,82.178574,35.694508,825.165405,35.850002,148,174,162,...,0.283748,0.163265,1.672265,62.625917,NaN,NaN,NaN,NaN,NaN,NaN
35949,195065314,11.712444,437.182220,34.533333,28.756390,632.724487,29.993334,127,155,137,...,0.009491,0.147462,1.895993,9.095304,0.005892,0.749986,0.000848,0.008639,0.137162,1.257774


In [ ]:
len(train_df.columns), len(test_df.columns)

(176, 164)

In [ ]:
len(train_df), len(test_df)

(55489, 6545)

In [ ]:
targets = [col for col in train_df.columns if col not in test_df.columns]
targets = targets[:6]

In [ ]:
metadata_columns = [col for col in train_df.columns if col in test_df.columns and col != 'id']

 TODO: hiányzó értékek kezelésének pontosítása

In [ ]:
# Check for missing values in the train DataFrame
train_missing_values = train_df.isnull().sum()
print("Missing values in Train DataFrame:")
print(train_missing_values)

Missing values in Train DataFrame:
id                                                                           0
WORLDCLIM_BIO1_annual_mean_temperature                                       0
WORLDCLIM_BIO12_annual_precipitation                                         0
WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month        0
WORLDCLIM_BIO15_precipitation_seasonality                                    0
                                                                         ...  
X11_sd                                                                   16341
X18_sd                                                                   16341
X26_sd                                                                   16341
X50_sd                                                                   16341
X3112_sd                                                                 16341
Length: 176, dtype: int64


In [ ]:
train_df.fillna(train_df.mean(), inplace=True)

# Getting the log of the targets

In [ ]:
columns_to_log = ['X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean',
                  'X4_sd', 'X11_sd', 'X18_sd', 'X26_sd', 'X50_sd', 'X3112_sd']

# Apply log10 transformation to the specified columns
for column in columns_to_log:
    train_df[column] = np.log10(train_df[column] + 1)  # Adding 1 to avoid log(0)


# Augmentation

In [ ]:
class AlbumentationsTransform:
    def __init__(self, augmentation):
        self.augmentation = augmentation

    def __call__(self, img):
        img_np = np.array(img)
        augmented = self.augmentation(image=img_np)
        return augmented['image']

train_augmentation =A.Compose([
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.25),
          A.Rotate(limit=45, p=0.4),
          A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), p=0.5),
          A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
          A.CoarseDropout(max_holes=6, max_height=8, max_width=8, min_holes=1, fill_value=0, p=0.5),
          A.Sharpen(alpha=0.3, lightness=0.3, p=0.5),
          A.Resize(224, 224)
        ])
# train_augmentation = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.Rotate(limit=45, p=0.5),
#     A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),
#     A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
#     # A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02, p=0.2),
#     # A.ElasticTransform(alpha=1, sigma=15, alpha_affine=15, p=0.2),
#     # A.GaussianBlur(blur_limit=3, p=0.5),
#     # A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
#     #A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=2, fill_value=0, p=0.5),
#     A.Sharpen(alpha=0.3, lightness=0.3, p=0.5),
#     # A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=50, p=0.5),
#     A.Resize(256, 256)
# ])

test_val_augmentation = A.Compose([
    A.Resize(224, 224) # ViT 224
])

final_transforms = transforms.Compose([
    AlbumentationsTransform(test_val_augmentation), #Ez miért kell?
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
class PlantTraits2024_Dataset(Dataset):
    def __init__(self, df, columns, images_path, augmentation=None,
                 transform=None, augment_times=1, metadata_columns=None,
                 augment_metadata=False,
                 metadata_augmentation_std=0.1, target = None):

        self.df = df
        self.columns = columns
        self.images_path = images_path
        self.augmentation = augmentation
        self.transform = transform
        self.augment_times = augment_times
        self.augment_metadata = augment_metadata
        self.metadata_augmentation_std = metadata_augmentation_std
        self.target = target

    def __len__(self):
        return len(self.df) * self.augment_times

    def __getitem__(self, idx) :
        original_idx = idx // self.augment_times
        id = self.df.iloc[original_idx, 0]
        image_path = f'{self.images_path}/{id}.jpeg'
        bgr_image = cv2.imread(image_path)
        image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        filtered_df = self.df[self.df['id'] == id].iloc[0]

        meta_data = torch.as_tensor(filtered_df[self.columns].to_numpy().astype(np.float64))

        if (self.augment_metadata):
          meta_data = self.augment_tabular_data(meta_data)

        if self.augmentation:
            image = AlbumentationsTransform(self.augmentation)(image)

        if self.transform:
            image = self.transform(image)

        # target
        #target_values = meta_data[self.target]
        #feature_values = meta_data[[col for col in meta_data.columns if col not in self.target]]
        return image, meta_data[:self.target[0]], meta_data[self.target[0]:]

    def augment_tabular_data(self, meta_data):
        noise = np.random.normal(0, self.metadata_augmentation_std, meta_data.shape)
        augmented_meta_data = meta_data # + noise
        return torch.as_tensor(augmented_meta_data, dtype=torch.float32)

    def get_original_image_and_metadata(self, idx):
        image_name = self.df.iloc[idx, 0]
        image_path = f'{self.images_path}/{image_name}.jpeg'
        bgr_image = cv2.imread(image_path)
        image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

        meta_data = self.df.iloc[idx][self.columns].to_numpy().astype(np.float64)
        meta_data = torch.tensor(meta_data, dtype=torch.float64)

        # target
        #target_values = meta_data[self.target]
        #feature_values = meta_data[[col for col in meta_data.columns if col not in self.target]]
        return image, meta_data[:self.target[0]], meta_data[self.target[0]:]


In [ ]:
def shorten_feature_name(name, max_length=15):
    if len(name) > max_length:
        return name[:max_length - 3] + "..."
    return name

def visualize_augmentations(dataset, idx, title, augment_times=4, n_features=5):
    features_to_display = dataset.columns[:n_features]
    original_image, meta_data, targets = dataset.get_original_image_and_metadata(idx)

    fig, axes = plt.subplots(1, augment_times + 1, figsize=(20, 4))
    fig.suptitle(title, fontsize=16)

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    original_image = np.array(original_image)
    axes[0].imshow(original_image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    for i in range(augment_times):
        augmented_image,  augmented_metadata, augmented_targets = dataset[idx * augment_times + i]
        augmented_image = augmented_image.permute(1, 2, 0).numpy()
        augmented_image = std * augmented_image + mean
        augmented_image = np.clip(augmented_image, 0, 1)
        axes[i + 1].imshow(augmented_image)
        axes[i + 1].set_title(f"Aug Version {i+1}")
        axes[i + 1].axis("off")

    plt.tight_layout()
    plt.show()

    header = ["Feature"] + ["Original"] + [f"Aug Version {i+1}" for i in range(augment_times)]
    header_format = "{:<30}" + "{:<15}" * (1 + augment_times)
    print(header_format.format(*header))

    data = torch.cat((meta_data, targets), -1)

    for feature in features_to_display:
        shortened_feature = shorten_feature_name(feature)
        row = [shortened_feature] + [f"{data[dataset.columns.index(feature)].item():.2f}"]
        for i in range(augment_times):
            _, augmented_metadata, augmented_targets = dataset[idx * augment_times + i]
            augmented_data = torch.cat((augmented_metadata, augmented_targets),-1)
            row.append(f"{augmented_data[dataset.columns.index(feature)].item():.2f}")
        print(header_format.format(*row))

In [ ]:
def split_dataframe(train_df, test_df, train_size=0.8, val_size=0.2):
    if train_size + val_size != 1:
        raise ValueError("train_size, val_size, and test_size must sum to 1")

    train_df, val_df = train_test_split(train_df, train_size=train_size, random_state=random_state)
    return train_df, val_df, test_df


In [ ]:
def create_dataset(train_df, test_df, columns, targets, train_images_path, test_images_path, augment_times, train_augmentation):
    # A képeknek csak a 10% át vesszük, hogy le tudjon futni Colabon a tanítás!!!
    train_df, val_df, test_df = split_dataframe(train_df.sample(frac=0.1, random_state=1), test_df.sample(frac=0.1, random_state=1))
    target = [i-1 for i, col in enumerate(train_df.columns) if col not in test_df.columns]
    target = target[:6]
    train_dataset = PlantTraits2024_Dataset(train_df, columns + targets, train_images_path, augmentation=train_augmentation, transform=final_transforms, augment_times=augment_times, augment_metadata=True,target=target)
    val_dataset = PlantTraits2024_Dataset(val_df, columns + targets, train_images_path, augmentation=test_val_augmentation, transform=final_transforms, augment_times=1, target=target)
    test_dataset = PlantTraits2024_Dataset(test_df, columns, test_images_path, augmentation=test_val_augmentation, transform=final_transforms, augment_times=1, target=target)

    return train_dataset, val_dataset, test_dataset

In [ ]:
train_dataset, val_dataset, test_dataset = create_dataset(train_df, test_df, metadata_columns, targets, TRAIN_IMAGES_PATH, TEST_IMAGES_PATH, augment_times=1, train_augmentation=train_augmentation)

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(4439, 1110, 654)

# Visualization

In [ ]:
#visualize_augmentations(dataset=train_dataset, title='Train dataset augmentations', idx=14, augment_times=2)

In [ ]:
#visualize_augmentations(dataset=val_dataset, title='Validation dataset augmentations', idx=14, augment_times=2)

In [ ]:
#visualize_augmentations(dataset=test_dataset, title='Test dataset augmentations', idx=14, augment_times=2)

#**Baselines**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

## Create dataloaders


In [ ]:
train_loader = DataLoader(train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size= 16, shuffle=False)

In [ ]:
# class PlantTraitsPretrainedEfficientNet(nn.Module):
#     def __init__(self):
#         super().__init__()

#         # Load a pre-trained EfficientNetB0 as the base model and remove the classifier
#         #self.model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights)
#         self.model = models.efficientnet_b0(pretrained=True)
#         self.model.classifier = nn.Identity()
#         self.fc = nn.Linear(1280, 6)  # 1280 is the number of features before the classifier in EfficientNetB0

#     def forward(self, x):
#         x = self.model(x)
#         x = self.fc(x)
#         return x

In [ ]:
class PlantTraitsPretrainedVGG16(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = models.vgg16(pretrained=True)
        # Replace last layer
        self.network.classifier = nn.Sequential(
           nn.Linear(in_features=25088, out_features=4096, bias=True),
           nn.ReLU(inplace=True),
           nn.Dropout(p=0.3, inplace=False),
           nn.Linear(in_features=4096, out_features=2048, bias=True),
           nn.ReLU(inplace=True),
           nn.Dropout(p=0.3, inplace=False),
           nn.Linear(in_features=2048, out_features=6, bias=True),
        )

    def forward(self, xb):
        return self.network(xb)

In [ ]:
# class PlantTraitsPretrainedResNet(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.network = models.resnet50(pretrained=True)
#         # Replace last layer
#         # self.network.classifier = nn.Sequential(
#            nn.Linear(in_features=self.network.fc.in_features, out_features=6, bias=True)
#         )

#     def forward(self, xb):
#         return self.network(xb)

In [ ]:
class PlantTraitsPretrainedVisionTransformer(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = models.vit_b_16(pretrained=True)

    self.network.heads.head = nn.Linear(self.network.heads.head.in_features, 6, bias=True)

  def forward(self, xb):
    return self.network(xb)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PlantTraitsPretrainedVGG16()
model = model.to(device)
model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 143MB/s]


PlantTraitsPretrainedVGG16(
  (network): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inpla

In [ ]:
!pip -q install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 12.3 MB/s eta 0:00:00


In [ ]:
from torchmetrics.regression import R2Score

In [ ]:
def r2_loss(output, target):
    target = target
    output = output
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return 1 - r2

In [ ]:
#Adatok GPU-ra való átmásolását végző függvény
def to_device(data, device):
  if isinstance(data, (list, tuple)):
    return [to_device(x, device) for x in data]
  return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader():
  def __init__(self, dl, device):
    self.dl = dl
    self.device = device

  def __iter__(self):
    for b in self.dl:
      yield to_device(b, self.device)

  def __len__(self):
    return len(self.dl)

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
from tqdm.notebook import tqdm #a progress bár esztétikus megjelenítésére

In [ ]:
def save_checkpoint(state, filename="/content/model_checkpoint.pth.tar"):
    torch.save(state, filename)
    print(f"Checkpoint saved to {filename}")

def load_checkpoint(checkpoint_file, model, optimizer, scheduler, device):
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint['epoch']

In [ ]:
class EarlyStoppingR2:
    def __init__(self, patience=10, delta=0, save=None):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.save = save

    def __call__(self, val_score, epoch, model, optimizer, scheduler):
        if self.best_score is None:
            self.best_score = val_score
            print(f"Best model loss: {val_score}")
            if self.save:
                checkpoint_filename = '/content/best_model.pth.tar'
                self.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                }, filename=checkpoint_filename)
        elif val_score < self.best_score - self.delta:
            self.counter += 1
            print(f"EarlyStopping patience counter: {self.counter}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if val_score > self.best_score:
                self.best_score = val_score
                self.counter = 0
                print(f"Best model loss: {val_score}")
                if self.save:
                    checkpoint_filename = '/content/best_model.pth.tar'
                    self.save({
                        'epoch': epoch + 1,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scheduler_state_dict': scheduler.state_dict(),
                    }, filename=checkpoint_filename)

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def train(model, train_loader, val_loader, criterion, max_lr, grad_clip=0, opt_func=torch.optim.Adam,  weight_decay=0, epochs=10,
          architecture_name= "CNN", dataset_ratio = 0.1, resume = False, checkpoint_path='/content/best_model.pth.tar', experiment_name="exp"):
    torch.cuda.empty_cache()
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    r2_score_metrics = R2Score(num_outputs=6).to(device)
     # set up one cycle lr scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,  steps_per_epoch=len(train_loader))
    early_stopping = EarlyStoppingR2(patience=3, save=save_checkpoint)

    wandb.init(
      # Set the project where this run will be logged
      project="Plant_traits",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=experiment_name,
      # Track hyperparameters and run metadata
      config={
        "max_learning_rate": max_lr,
        "architecture": architecture_name,
        "epochs": epochs,
        "dataset_ratio" : dataset_ratio,
        "criterion" : criterion,
        "grad_clip" : grad_clip,
        "opt_func" : opt_func
      })

    start_epoch = 0
    if resume and checkpoint_path:
        start_epoch = load_checkpoint(checkpoint_path, model, optimizer, sched, device)

    best_loss = float('inf')
    for epoch in range(start_epoch, start_epoch + epochs):
        model.train()
        epoch_loss = 0
        lrs = []
        for images, meta_data, targets in tqdm(train_loader):


            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            if grad_clip:
               nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item() * images.size(0)
            lrs.append(get_lr(optimizer))
            sched.step()

        epoch_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

        model.eval()  # Set model to evaluation mode

        with torch.no_grad():
            epoch_val_loss = 0.0
            r2_epoch_score = 0.0
            for images, meta_data, targets in tqdm(val_loader):

                outputs = model(images)
                loss = criterion(outputs, targets)
                epoch_val_loss += loss.item() * images.size(0)
                r2_epoch_score += r2_score_metrics(outputs, targets)

            epoch_val_loss = epoch_val_loss /len(val_loader)
            r2_score = r2_epoch_score / len(val_loader)


            print(f'Epoch {epoch + 1}/{epochs} Validation Loss: {epoch_val_loss:.4f}')
            print(f'Validation R2 Score: {r2_score}')
            wandb.log({"val_loss": epoch_val_loss, "loss": epoch_loss, "r2_score": r2_score})
            early_stopping(r2_score, epoch, model, optimizer, sched)
            if early_stopping.early_stop:
                print("Early stopping triggered.")
                break

    # Mark the run as finished
    wandb.finish()

def test_model(model, criterion, test_loader):
    model.eval()  # Set model to evaluation mode
    epoch_loss = 0.0
    r2_score_metrics = R2Score(num_outputs=6).to(device)
    r2_epoch_score = 0
    with torch.no_grad():
       for images, meta_data, targets in test_loader:
            outputs = model(images)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item() * images.size(0)
            r2_epoch_score += r2_score_metrics(outputs, targets)

    test_loss = epoch_loss / len(test_loader)
    r2_score = r2_epoch_score / len(val_loader)
    print(f'Test Loss: {test_loss:.4f}')
    print(f'Validation R2 Score: {r2_score}')


Train a
- ResNet (too slow)
- VGG
- Efficientnet

In [ ]:
epochs = 10
opt_func = torch.optim.SGD
loss_func =r2_loss #  nn.MSELoss()
max_lr = 0.01
grad_clip = 0
weight_decay = 1e-6
dataset_ratio = 0.1
architecture_name = "CNN - EfficientNetB0 - None"

In [ ]:
train(model, train_loader, val_loader, criterion = loss_func, max_lr = max_lr, grad_clip=0, opt_func=opt_func, weight_decay= weight_decay, epochs=epochs,
      architecture_name= architecture_name,dataset_ratio = dataset_ratio, experiment_name = "ViT - None" )

  0%|          | 0/833 [00:00<?, ?it/s]

Epoch 1, Loss: 3.551222565723639


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1/10 Validation Loss: 2.9688
Validation R2 Score: -0.020987868309020996
Best model loss: -0.020987868309020996
Checkpoint saved to /content/best_model.pth.tar


  0%|          | 0/833 [00:00<?, ?it/s]

Epoch 2, Loss: 2.792945068280379


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 2/10 Validation Loss: 3.0112
Validation R2 Score: 0.0716974139213562
Best model loss: 0.0716974139213562
Checkpoint saved to /content/best_model.pth.tar


  0%|          | 0/833 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test_model(model, r2_loss, val_loader)

Test Loss: 3.4380
Validation R2 Score: -0.08563786000013351


In [ ]:
from enum import Enum

class AugmentationsComplexity(Enum):
  NONE = {
      "augmentation" : None,
      "augmentation_times" : 1,
  }
  SMALL = {
      "augmentation" : A.Compose([
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.5),
          A.Rotate(limit=45, p=0.5),
          A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),
          A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
          A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=2, fill_value=0, p=0.5),
          A.Sharpen(alpha=0.3, lightness=0.3, p=0.5),
          A.Resize(256, 256)
        ]),
      "augmentation_times" : 3,
  }
  LARGE = {
      "augmentation" : A.Compose([
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.5),
          A.Rotate(limit=45, p=0.5),
          A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),
          A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
          A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02, p=0.2),
          A.ElasticTransform(alpha=1, sigma=15, alpha_affine=15, p=0.2),
          A.GaussianBlur(blur_limit=3, p=0.5),
          A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
          A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=2, fill_value=0, p=0.5),
          A.Sharpen(alpha=0.3, lightness=0.3, p=0.5),
          A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=50, p=0.5),
          A.Resize(256, 256)
        ]),
      "augmentation_times" : 3,
  }

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/blur/transforms.py:184: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [ ]:
from enum import Enum

class AugmentationsComplexity(Enum):
  NONE = {
      "augmentation" : None,
      "augmentation_times" : 1,
  }
  SMALL = {
      "augmentation" : A.Compose([
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.5),
          A.Rotate(limit=45, p=0.5),
          A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),
          A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
          A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=2, fill_value=0, p=0.5),
          A.Sharpen(alpha=0.3, lightness=0.3, p=0.5),
          A.Resize(256, 256)
        ]),
      "augmentation_times" : 3,
  }
  LARGE = {
      "augmentation" : A.Compose([
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.5),
          A.Rotate(limit=45, p=0.5),
          A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),
          A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
          A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02, p=0.2),
          A.ElasticTransform(alpha=1, sigma=15, alpha_affine=15, p=0.2),
          A.GaussianBlur(blur_limit=3, p=0.5),
          A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
          A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=2, fill_value=0, p=0.5),
          A.Sharpen(alpha=0.3, lightness=0.3, p=0.5),
          A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=50, p=0.5),
          A.Resize(256, 256)
        ]),
      "augmentation_times" : 3,
  }

In [ ]:
checkpoint_path = '/content/best_efficientnetv2'
experiment_name =  'efficientNetv2'

In [ ]:
def train_with_different_augmentations():
  for complexity in [AugmentationsComplexity.NONE,AugmentationsComplexity.SMALL,AugmentationsComplexity.LARGE]:
    train_dataset, val_dataset, test_dataset = create_dataset(train_df, test_df, metadata_columns, targets, TRAIN_IMAGES_PATH, TEST_IMAGES_PATH, augment_times=complexity.value["augmentation_times"], train_augmentation=complexity.value["augmentation"])
    print(len(train_dataset), len(val_dataset), len(test_dataset))
    train_loader = DataLoader(train_dataset, batch_size= 16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = False)
    test_loader = DataLoader(test_dataset, batch_size= 16, shuffle=False)
    train_loader = DeviceDataLoader(train_loader, device)
    val_loader = DeviceDataLoader(val_loader, device)
    test_loader = DeviceDataLoader(test_loader, device)

    train(model, train_loader, val_loader, criterion = loss_func, max_lr = max_lr, grad_clip=grad_clip, opt_func=opt_func, weight_decay=weight_decay, epochs=10,
        architecture_name="EfficientNet", dataset_ratio = dataset_ratio, resume=False, checkpoint_path=f'{checkpoint_path}_{complexity.name}.pth.tar', experiment_name=f'{experiment_name}_{complexity.name}')

In [ ]:
train_with_different_augmentations()

4439 1110 654


  0%|          | 0/278 [00:00<?, ?it/s]

Epoch 1, Loss: 0.8874565026314138


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1/10 Validation Loss: 3.2838
Validation R2 Score: -0.0012941871536895633
Best model loss: -0.0012941871536895633
Checkpoint saved to /content/best_model.pth.tar


...

  0%|          | 0/833 [00:00<?, ?it/s]

Epoch 5, Loss: 0.9598073235222129


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 5/10 Validation Loss: 3.3829
Validation R2 Score: -0.05625544488430023
EarlyStopping patience counter: 1


  0%|          | 0/833 [00:00<?, ?it/s]

# Hyperopt

In [ ]:
def train(model, train_loader, val_loader, criterion, max_lr, opt_func, grad_clip=0, weight_decay=0, epochs=10,
          architecture_name="CNN", dataset_ratio=0.1, resume=False, checkpoint_path='/content/best_model.pth.tar', experiment_name="exp", trial=None):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    torch.cuda.empty_cache()
    optimizer = opt_func(model.parameters())
    r2_score_metrics = R2Score(num_outputs=6).to(device)
    sched = OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))

    wandb.init(
        project="Plant_traits",
        name=experiment_name,
        config={
            "max_learning_rate": max_lr,
            "architecture": architecture_name,
            "epochs": epochs,
            "dataset_ratio": dataset_ratio,
            "weight_decay": weight_decay,
            "criterion": criterion,
            "grad_clip": grad_clip,
            "optimizer": optimizer
        }
    )

    start_epoch = 0
    # if resume and checkpoint_path:
    #     start_epoch = load_checkpoint(checkpoint_path, model, sched, device, optimizer)

    best_loss = float('inf')
    for epoch in range(start_epoch, epochs):
        model.train()
        total_loss = 0
        lrs = []

        for images, meta_data, targets in tqdm(train_loader):
            images, targets = images.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            if grad_clip:
                torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            sched.step()

            total_loss += loss.item() * images.size(0)
            lrs.append(get_lr(optimizer))

        epoch_loss = total_loss / len(train_loader)
        wandb.log({"epoch": epoch, "loss": epoch_loss, "lr": lrs[-1]})

        model.eval()
        epoch_val_loss, r2_epoch_score = 0.0, 0.0
        with torch.no_grad():
            for images, meta_data, targets in tqdm(val_loader):
                images, targets = images.to(device), targets.to(device)
                outputs = model(images)
                loss = criterion(outputs, targets)
                epoch_val_loss += loss.item() * images.size(0)
                r2_epoch_score += r2_score_metrics(outputs, targets)

        epoch_val_loss /= len(val_loader)
        r2_score = r2_epoch_score / len(val_loader)

        wandb.log({"val_loss": epoch_val_loss, "loss": epoch_loss, "r2_score": r2_score})
        print(f'Epoch {epoch + 1}/{epochs} Validation Loss: {epoch_val_loss:.4f}')
        print(f'Validation R2 Score: {r2_score}')

        if trial:
            trial.report(epoch_val_loss, epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()

    wandb.finish()
    return epoch_val_loss

In [ ]:
def objective(trial):
    max_lr = trial.suggest_float('max_lr', 1e-4, 1e-2)

    optimizer_choices = {'Adam': optim.Adam, 'SGD': optim.SGD}
    optimizer_name = trial.suggest_categorical('optimizer', list(optimizer_choices.keys()))
    optimizer_class = optimizer_choices[optimizer_name]

    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-4)
    grad_clip = trial.suggest_float('grad_clip', 0, 0.6)

    if optimizer_name == 'SGD':
        momentum = trial.suggest_float('momentum', 0.7, 1.0)
        optimizer = lambda params: optimizer_choices[optimizer_name](params, lr=max_lr, momentum=momentum, weight_decay=weight_decay)
    else:
        optimizer = lambda params: optimizer_choices[optimizer_name](params, lr=max_lr, weight_decay=weight_decay)


    model = PlantTraitsPretrainedVGG16()
    model = model.to(device)

    return train(model, train_loader, val_loader, r2_loss, max_lr, optimizer, grad_clip, weight_decay, epochs = 5, trial=trial, experiment_name=f'Vgg16 - hyperopt_2')

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=15)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-21 08:56:57,341] A new study created in memory with name: no-name-54c9b7fc-904f-482d-9e25-33b6cc0ef379
wandb: Currently logged in as: tugyibeatrix (beateam). Use `wandb login --relogin` to force relogin


  0%|          | 0/278 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1/5 Validation Loss: 27.7993
Validation R2 Score: -10.125091552734375


  0%|          | 0/278 [00:00<?, ?it/s]

...

Epoch 2/5 Validation Loss: 3.2700
Validation R2 Score: -0.009474314749240875


  0%|          | 0/278 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

[I 2024-04-21 10:32:08,446] Trial 14 pruned. 


Epoch 3/5 Validation Loss: 3.2343
Validation R2 Score: -0.029823238030076027
Best trial:
  Value: 3.0285968450263394
  Params: 
    max_lr: 0.00749356550204518
    optimizer: SGD
    weight_decay: 4.445895279156939e-06
    grad_clip: 0.43926232579524443
    momentum: 0.9868775854883227


In [ ]:
fgffhffffdbbb

In [ ]:
def objective(trial):
    max_lr = trial.suggest_float('max_lr', 1e-4, 1e-2)

    optimizer_choices = {'Adam': optim.Adam,  'SGD': optim.SGD}
    optimizer_name = trial.suggest_categorical('optimizer', list(optimizer_choices.keys()))
    optimizer_class = optimizer_choices[optimizer_name]

    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-4)
    grad_clip = trial.suggest_float('grad_clip', 0, 0.6)

    if optimizer_name == 'SGD':
        momentum = trial.suggest_float('momentum', 0.7, 1.0)
        optimizer = lambda params: optimizer_choices[optimizer_name](params, lr=max_lr, momentum=momentum, weight_decay=weight_decay)
    else:
        optimizer = lambda params: optimizer_choices[optimizer_name](params, lr=max_lr, weight_decay=weight_decay)


    model = PlantTraitsPretrainedVisionTransformer()
    model = model.to(device)

    return train(model, train_loader, val_loader, r2_loss, max_lr, optimizer, grad_clip, weight_decay, epochs = 5, trial=trial, experiment_name='ViT - hyperopt')

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=8)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-16 06:04:21,717] A new study created in memory with name: no-name-ba7f248c-75a1-4a36-89e9-5e1b86fde641
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: Currently logged in as: tugyibeatrix (beateam). Use `wandb login --relogin` to force relogin


  0%|          | 0/555 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch 1/5 Validation Loss: 2.7746
Validation R2 Score: 0.02474934048950672


  0%|          | 0/555 [00:00<?, ?it/s]

...

Epoch 1/5 Validation Loss: 2.8289
Validation R2 Score: -0.04341914504766464


  0%|          | 0/555 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch 2/5 Validation Loss: 2.6058
Validation R2 Score: 0.10019311308860779


  0%|          | 0/555 [00:00<?, ?it/s]

#1. Proba:

In [ ]:
def objective(trial):
    max_lr = trial.suggest_float('max_lr', 1e-4, 1e-2)

    optimizer_choices = {'Adam': optim.Adam,  'SGD': optim.SGD}
    optimizer_name = trial.suggest_categorical('optimizer', list(optimizer_choices.keys()))
    optimizer_class = optimizer_choices[optimizer_name]

    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-4)
    grad_clip = trial.suggest_float('grad_clip', 0, 0.6)

    if optimizer_name == 'SGD':
        momentum = trial.suggest_float('momentum', 0.7, 1.0)
        optimizer = lambda params: optimizer_choices[optimizer_name](params, lr=max_lr, momentum=momentum, weight_decay=weight_decay)
    else:
        optimizer = lambda params: optimizer_choices[optimizer_name](params, lr=max_lr, weight_decay=weight_decay)


    model = PlantTraitsPretrainedVisionTransformer()
    model = model.to(device)

    return train(model, train_loader, val_loader, r2_loss, max_lr, optimizer, grad_clip, weight_decay, epochs = 5, trial=trial, experiment_name='ViT - hyperopt')

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=8)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-15 21:01:14,174] A new study created in memory with name: no-name-fe87f8e6-83d8-4e73-8166-b243c2c569a5


  0%|          | 0/555 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch 1/5 Validation Loss: 3.4136
Validation R2 Score: -0.0352247916162014


  0%|          | 0/555 [00:00<?, ?it/s]

...

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch 5/5 Validation Loss: 2.5013
Validation R2 Score: 0.15790128707885742


epoch,▁▃▅▆█
loss,██▅▅▄▄▃▃▁▁
lr,▇█▅▂▁
r2_score,▁▄▃▇█
val_loss,█▅▅▂▁
epoch,5
loss,2.48966
lr,0.0
r2_score,0.1579
val_loss,2.50134


[I 2024-04-16 00:15:33,070] Trial 4 finished with value: 2.5013350391119107 and parameters: {'max_lr': 0.02686653866193137, 'optimizer': 'SGD', 'weight_decay': 0.0006261871659442823, 'grad_clip': 0.6119716478862998, 'momentum': 0.9185932354903407}. Best is trial 4 with value: 2.5013350391119107.


Best trial:
  Value: 2.5013350391119107
  Params: 
    max_lr: 0.02686653866193137
    optimizer: SGD
    weight_decay: 0.0006261871659442823
    grad_clip: 0.6119716478862998
    momentum: 0.9185932354903407
